# Regressione multipla e regressione multivariata

I minimi quadrati visti nell'esempio precedente sono un esempio di **regressione**.
L'obiettivo della analisi di regressione è quello di descrivere una relazione tra una o più variabili dette variabili dipendenti e un altro insieme di variabili dette indipendenti o esplicative.

Nel capitolo precendente abbiamo usato l'età della madre come variabile indipendente per fare una previsione sul peso del nascituro come variabile dipendente. Quando c'è una sola variabile dipendente e una sola variabile indipendente si dice che siamo di fronte ad un problema di semplice regressione. In questo capitolo ci muoveremo verso la regressione multipla che tratta più di una variabile esplicativa.

Se invece si tratta di avere più variabili dipendenti allora possiamo dire di trattare un problema di regressione multivariata.
Se la relazione tra le variabili dipendenti e le variabili indipendenti è lineare allora diremo che la regressione è lineare.

Per esempio se la variabile $y$ è la **variabile dipendente** mentre $x_1$ e $x_2$ sono le **variabili esplicative** potremo scrivere il nostro modello come:

$$
    y = \beta_0 + x_1 \beta_1 + x_2 \beta_2 + \varepsilon
$$

dove $\beta_0$ rappresenta il punto di intersezione mentre $\beta_1$ e $\beta_2$ rappresentano i parametri associati alle variabili e $\varepsilon$ rappresenta il residuale dovuto a fattori casuali o sconosciuti.

L'obiettivo è far si che $\beta_0$,$\beta_1$ e $\beta_2$ minimizzino $\varepsilon^2$, questo processo è anche chiamato metodo dei minimi quadrati **ordinary least squares**.

## Statsmodels

Per una regressione singola abbiamo usato il codice visto in precenza per creare del codice capibile. Ora dovremo utilizzare un pacchetto più completo chiamato [statsmodels](https://www.statsmodels.org/stable/index.html).

Se usiamo anaconda come sto facendo io questo pacchetto è presente altrimenti va installato.<br/>
Qui sotto un esempio di codice

In [43]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from utils import readReadFemPreg

preg = readReadFemPreg()
live = preg[preg.outcome == 1]

live = live.dropna(subset=['agepreg','totalwgt_lb'])

formula = 'totalwgt_lb ~ agepreg'
model = smf.ols(formula, data=live)
results = model.fit()

`statsmodels` fornisce due interfacce le api formula che usano stringhe per identificare i due tipi di variabili dipendenti e indipendenti. Viene usato il linguaggio [`patsy`](https://patsy.readthedocs.io/en/latest/). E il simbolo `~` identifica la separazione tra le variabili dipendenti sulla sinistra e quelle indipendenti sulla destra.

Viene poi usata la parte `smf.ols` che lavora la string e il dataframe `live` e restituisce un oggetto OLS che rappresenta il modello come detto prima OLS sta per "ordinary least squares" metodo dei minimi quadrati. 

Il metodo `fit` adatta (fit) il modello ai dati e ci restituice un oggetto `RegressionResults` che contiene il risultato voluto.

I risultati sono disponibili come attributi. Params è una Serie che mappa i nomi delle variabili con i loro parametri, otteniamo dunque quello che ci serve.

In [44]:
inter = results.params['Intercept']
slope = results.params['agepreg']

print("intercept = {0:.2f} slope = {1:.4f}".format(inter,slope))

intercept = 6.83 slope = 0.0175


I valori sono in linea a quanto ottenuto nel capitolo precedente.
Un altro attributo interessante è `pvalues` che mappa i nomi delle variabili con il loro p-value, possiamo dunque verificare se il parametro slope (pendenza) è statisticamente significante.

In [45]:
slope_pvalue = results.pvalues['agepreg']
slope_pvalue

5.722947107314471e-11

Vediamo che il valore è molto minore di 0.001 come ci può aspettare. 
Il prossomo campo è `results.rsquared` che contiene $R^2$ che ha come valore 0.0047, `results` ci fornisce anche `f_pvalue` che è il valore p-value associato al modello nel suo complesso che assomiglia a verificare se $R^2$ è statisticamente significante.
Abbiamo che il campo `resid` una sequenza di residuali e `fittedvalues` una sequenza di valori previsti dal modello usando `agepreg`.

In [46]:
rsquared = results.rsquared 
f_pvalue = results.f_pvalue

print("rsquared = {0:.4f} f_pvalue = {1:.4f}".format(rsquared,f_pvalue))

rsquared = 0.0047 f_pvalue = 0.0000


In [47]:
resid = results.resid 
fittedvalues = results.fittedvalues

print(resid[:10])

0    1.403333
1    0.359539
2    2.044489
3   -0.141599
4   -0.962826
5    1.260849
6    2.228908
7    1.018195
8    0.241999
9   -0.769680
dtype: float64


In [48]:
print(fittedvalues[:10])

0    7.409167
1    7.515461
2    7.080511
3    7.141599
4    7.150326
5    7.301651
6    7.333592
7    7.356805
8    7.320501
9    7.394680
dtype: float64


L'oggetto results ci fornisce anche il comodo summary(), che rappresenta i risultati in un formato leggbile.

In [49]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Thu, 10 Feb 2022   Prob (F-statistic):           5.72e-11
Time:                        10:04:47   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.0

## Regressione multipla

Nelle lezioni precedenti abbiamo detto che il primo bambino tende ad essere più leggero dei successivi, e questo effetto è statisticamente significante. 

Ma questo è uno strano risultato perchè non abbiamo una spiegazione ovvia sul perchè, in effetti una possibile spiegazione potrebbe esserci, abbiamo visto che il peso del bambino è legato alla età della madre e potremmo dunque aspettarci che le madri che partoriscono il primo figlio siano più giovani delle altre.

Con qualche calcolo verificheremo se questa spiegazione, useremo quindi la regressione multipla per indagare più attentamente.
Per prima cosa vediamo quanto è grande la differenza di peso.

In [50]:
firsts = live[live.birthord == 1]
others = live[live.birthord > 1]


diff_weight = firsts.totalwgt_lb.mean() - others.totalwgt_lb.mean()
diff_weight

-0.12476118453549034

Vediamo poi la differenza di età tra il primo figlio e gli altri

In [51]:
diff_age = firsts.agepreg.mean() - others.agepreg.mean()
diff_age

-3.558721516986065

e calcoliamo quando dovrebbe essere la differenza di peso usando il modello di regressione creato prima

In [52]:
slope * diff_age

-0.06211339678698344

si vede che la differenza di peso previstà è esattamente la metà di quella media, quindi possiamo concludere almeno in parte che l'età della madre spiega l'aumento del peso solo in parte.

Usando la regressione multipla, possiamo esplorare queste relazioni in modo più sistematico.

In [53]:
live['isfirst'] = live.birthord == 1
formula = 'totalwgt_lb ~ isfirst'
results = smf.ols(formula, data=live).fit()

La prima riga del codice crea una nuova colonna che ha valore `True` se stiamo vedendo il primo figlio `False` altrimenti.
Di seguito adattiamo il modello usando la colonna `isfirst` come variabile indipendente.

In [54]:
def summarizeResults(results):
    """
    visualizza i parametri più importanti del modello di regressione
    """
    for name, param in results.params.items():
        pvalue = results.pvalues[name]
        print('%s   %0.3g   (%.3g)' % (name, param, pvalue))

    try:
        print('R^2 %.4g' % results.rsquared)
        ys = results.model.endog
        print('Std(ys) %.4g' % ys.std())
        print('Std(res) %.4g' % results.resid.std())
    except AttributeError:
        print('R^2 %.4g' % results.prsquared)
        
summarizeResults(results)

Intercept   7.33   (0)
isfirst[T.True]   -0.125   (2.55e-05)
R^2 0.00196
Std(ys) 1.408
Std(res) 1.407


Il parametro `isfirst` è un booleano, `ols` lo tratta come una variabile categorica e non può essere trattato come un numero.
Il parametro stimato è l'effetto sul peso del nascituro quando `isfirst` ha valore `True` il risultato è -0.125 che rappresenta la differenza di peso tra il primo e gli altri bambini.

Il risultato è statisticamente significativo il che significa che è improbabile che la relazione si verifichi per caso, ma il valore $R^2$ risultante è molto piccolo ($R^2$ assume valori compresi tra 0 e 1) il che significa che la variabile `isfirst` 
non rappresenta una parte sostanziale nella variazione del peso del nascituro.

Stesso ragionamento vale per `agepreg` che come abbiamo visto prima ha un valore $R^2$ pari a 0.0047
Proviamo ora ad assemblare un modello che includa entrambe le variabili.
Con la formula `totalwgt_lb ~ isfirst + agepreg` otteniamo:

In [55]:
formula = 'totalwgt_lb ~ isfirst + agepreg'
results = smf.ols(formula, data=live).fit()

summarizeResults(results)

Intercept   6.91   (0)
isfirst[T.True]   -0.0698   (0.0253)
agepreg   0.0154   (3.93e-08)
R^2 0.005289
Std(ys) 1.408
Std(res) 1.405


Nel nuovo modello, i parametri per `isfirst` sono piccoli di circa la metà, il che significa che la parte dell'effetto apparente di `isfirst` è recuperata da `agepreg`. Anche il p-value di `isfirst` è circa il 2.5% che è al confine della significatività statistica.

Il valore $R^2$ di questo modello è leggermente maggiore, il che indica che le due variabili assieme contribuiscono a migliorare la previsione del peso (ma non di molto).

## Relazioni non lineari

Ricordando che il contributo di `agepreg` potrebbe essere non lineare, potremmo considerare una variabile per catturare meglio questa relazione. Una opzione è quella di creare una colonna `agepreg2` valorizzata con il quadrato delle età.


In [56]:
live['agepreg2'] = live.agepreg**2
formula = 'totalwgt_lb ~ isfirst + agepreg + agepreg2'
results = smf.ols(formula, data=live).fit()
summarizeResults(results)

Intercept   5.69   (1.38e-86)
isfirst[T.True]   -0.0504   (0.109)
agepreg   0.112   (3.23e-07)
agepreg2   -0.00185   (8.8e-06)
R^2 0.007462
Std(ys) 1.408
Std(res) 1.403


Ora usando i parametri `agepreg` e `agepreg2` stiamo effettivamente costruendo una parabola. Il valore di `agepreg2` è negativo il che vuol dire che la parabola curva verso il basso il che concorda con quanto visto precedentemente.

Il modello quadratico gestisce meglio la variabilità del peso e sempre in questo modello il parametro `isfirst` è piccolo e sempre meno statistimanente significativo. 

L'uso delle variabili calcolate come `agepreg2` è un  modo comune per adattare modelli polinomiali e altre funzioni ai dati.

Il processo può essere ancora considerato una regressione lineare perchè la variabile dipendente è una funzione lineare delle variabili indipendenti, indipendentemente dal fatto che alcune variabili siano funzioni non lineari di altre.

La seguente tabellina indica i risultati delle regressioni:

|           	|  isfirst 	        |   agepreg	    |   agepreg2	|   R2	    |
|:-:	        |---	            |---	        |---	        |---	    |
|   Modello 1	|    -0.125 *	    |   –	        |   –	        |   0.002	|
|   Modello 2	|   –	            |   0.0175 *	|   –	        |   0.0047	|
|   Modello 3	|   -0.0698 (0.025)	|   0.0154 *	|   –        	|   0.0053	|
|   Modello 4	|   -0.0504 (0.11)	|   0.112 *     |   -0.00185 *	|   0.0075	|


Le colonne in questa tabella sono le variabili esplicative e i lori coefficienti $R^2$, Ogni riga rappresenta il parametro stimato e il suo p-value nelle i valori asteriscati cono quelli che hanno un p-value minore dello 0.001.

Possiamo concludere che l'apparente differenza di peso è spiegato, almeno in parte, dalla differenza di età della madre.
Quando si include l'età nel modello l'effetto della variabile `isfirst` diventa più piccolo.

In questo esempio, l'età della madre funge da variabile di controllo. L'inclusione di `agepreg` nel modello "controlla" la differenza in età tra le prime mamme e le altre, rendendo così possibile isolare l'effetto (se mai ci fosse) di `isfirst`.


## Data mining

Abbiamo appena usato un modello di regressione per dare delle spiegazioni, per esempio nella sezione precedente abbiamo scoperto che una differenza apparente nel peso del bambino è legato alla differenza di età della madre.
Ma il valore di $R^2$ di questo modello è molto basso, il che significa che ha uno scarso potere predittivo. In questa sezione proveremo a fare di meglio. 

Supponiomo che una delle nostre colleghe aspetti un bambino e che in ufficio si stia scommettendo sul peso del nascituro. 
Ora supponiamo che tu voglia veramente vincere la scommessa. Come si potrebbe aumentare la sperannza di vittoria ?
Bene il dataset contiene 244 variabili per ognuna delle gravidanze e altre 3087 vaiabili per ciascun intervistato.
Potrebbe essere che qualcuna di queste variabili abbia un potere predittivo. Per scoprire le più utili perche non provarle tutte?.

Provare le variabili della tabella delle gravidanze è semplice, ma usare quella degli intervistati risulta un pochino più complicato. In teoria possiamo iterare attraverso le righe della tabella delle gravidanze, usare il campo `caseid` per trovare le risposte dell'intervistato e copiare i valori nella tabella gravidanze.
Questo però risulta un po lento.

Una soluzione migliore è quella di usare l'operazione di join messa a disposizione dall'oggetto DataFrame:

In [57]:
from utils import readFemResp

live = live[live.prglngth>30]
resp = readFemResp()

resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

La prima riga seleziona i record delle gravidanze più lunghe di 30 settimane, stiamo assumendo che la scommessa sia stata fatta alcune settimane prima dell'evento.

Infine in questo esempio alcune colonne sono presenti sia nella prima che nella seconda tabella ad esempio la colonna `race` darà come risultato due colonne `race` e `race_r`.

Proviamo ora le variabili:

In [58]:
 t = []
for name in join.columns:
    try:
        if join[name].var() < 1e-7:
            continue

        formula = 'totalwgt_lb ~ agepreg + ' + name
        model = smf.ols(formula, data=join)
        if model.nobs < len(join)/2:
            continue

        results = model.fit()
    except (ValueError, TypeError):
        continue

    t.append((results.rsquared, name))

Per ogni variabile costruiamo il modello, calcoliamo $R^2$ e mettiamo il risultato nella lista. Tutti i modelli includono `agepreg` che sappiamo già avere un potere predittivo.

Controlliamo ogni variabile indipendente abbia una certa variabilità, altrimenti il risultato della regressione sarà inaffidabile. 
Controlliamo anche il numero delle osservazioni per ogni modello. Variabili che contengono un gran numero di NAN non sono dei buoni candidati per fare una previsione.

Per molte di queste variabili, non abbiamo fatto nessuna attività di pulizia. Alcune di queste sono codificate in un modo che non lavora molto bene per la regressione lineare.
Come risultato, potremmo escludere alcune variabili che sarebbero utili se pulite correttamente. Ma forse troveremo lo stesso dei buoni candidati.

## Previsioni
Il prossimo passo è quello di ordinare i risultati e selezionare le variabiali che producono i più alti valori di $R^2$.

In [59]:
t.sort(reverse=True)
for mse, name in t[:30]:
    print(name, mse)

totalwgt_lb 1.0
birthwgt_lb 0.9498127305978009
lbw1 0.3008240784470769
prglngth 0.13012519488625063
wksgest 0.12340041363361054
agecon 0.10203149928156041
mosgest 0.02714427463957969
babysex 0.01855092529394209
race_r 0.016199503586252995
race 0.016199503586252995
nbrnaliv 0.016017752709788113
paydu 0.014003795578114597
rmarout03 0.013430066465713209
birthwgt_oz 0.013102457615706165
anynurse 0.012529022541810764
bfeedwks 0.01219368840449575
totincr 0.011870069031173602
marout03 0.011807801994375033
marcon03 0.011752599354395654
cebow 0.011437770919637047
rmarout01 0.011407737138640184
rmarout6 0.011354138472805753
marout01 0.011269357246806555
hisprace_r 0.011238349302030715
hisprace 0.011238349302030715
mar1diss 0.010961563590751733
fmarcon5 0.0106049646842995
rmarout02 0.0105469132065652
marcon02 0.01048140179553414
fmarout5 0.010461691367376957


Le prime due variabili sono `totalwgt_lb` e `birthwgt_lb` che ovviamente non possono essere usate per prevedere il peso del bambino.

Similmente `prglngth` ha un buon potere predittivo, ovviamente per la nostra scommesssa la gestazione non può essere usata.
La prima variabile utile è la `babysex` che indica se il bambino è un maschietto o una femminuccia. 
Nel nostro dataset i ragazzi sono all'incirca 0.3 lbs più pesanti. 
Assumendo di sapere il sesso del bimbo, possiamo usarla per la previsone.

Il prossimo campo è la razza, indicante se l'intervistato è bianco nero o altro. Come variabile esplicativa, la razza può essere problematica. In dataset come NSFG la razza è correlata con moltre altre variabili, incluso reddito e altri fattori socioeconomici.

In un modello di regressione, la razza si comporta come una variabile prox, dunque le correlazioni del campo razza sono spesso causatel almeno in parte da altri fattori.

La prossima variabile nella lista è `nbrnaliv`, che indica se la gravidanza ha prodotto dei gemelli. I gemelli tendono ad essere più leggeri degli altri bambini, dunque per la nostra scommessa questo può aiutare.

Andiamo ora a vedere la variabile `paydu`, che indica se l'intervistato possiede la sua casa. E' una delle tante relative variabili relativi al reddito che risultano predettivi. In questo dataset reddito e salute sono correlati praticamente a tutto.
In questo esempio il reddito, la dieta, l'assistenza sanitaria e altri fattori sono correlati al peso del bebè.

Alcune altre variabili nella lista sono cose che non possiamo usare, come ad esempio `bfeedwks` il numero di settimane che il bambino è stato allattato al seno.
Non possiamo usare queste variabili per la nostra previsione, ma possiamo provare a fare delle speculazioni.
A volte si inizia con una teoria e si usano i dati per confermarla, altre volte si inizia con i dati e si va alla ricerca di qualche teoria.

Questo approccio è chiamato **data mining**. Un vantaggio è quello di poter scoprire delle relazioni inaspettate. Un rischio però 
è che molte delle relazioni scoperte siano randomiche o spurie.
Avendo identificato le potenziali varibili indipendenti scegliamo questo modello:

In [60]:
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 + '
               'nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=join).fit()

Questa formula usa della sintassi che non abbiamo mai visto prima: 

* `C(race)` dice al parser di trattare la razza come una variabile categorica anche se codificata come un numero.
* La codifica di `babysex` è 1 per i maschietti 2 per le femminucce, scrivendo `babysex==1` questo converte il campo in un booleano 
* Similmente `nbrnaliv>1` è True per i parti gemellari mentre `paydu==1` per gli intervistati che hanno la loro casa.
* `totincr` è codificato come un numero compreso tra 1 e 14 ogni incremento rappresenta circa 5000$ di inclemento nel reddito annuo. Possiamo dunque trattare questa variabile come numerica. 

Questo il risultato:

In [61]:
summarizeResults(results)

Intercept   6.63   (0)
C(race)[T.2]   0.357   (5.43e-29)
C(race)[T.3]   0.266   (2.33e-07)
babysex == 1[T.True]   0.295   (5.39e-29)
nbrnaliv > 1[T.True]   -1.38   (5.1e-37)
paydu == 1[T.True]   0.12   (0.000114)
agepreg   0.00741   (0.0035)
totincr   0.0122   (0.00188)
R^2 0.05999
Std(ys) 1.271
Std(res) 1.232


Abbiamo avuto in incremento di $R^2$ ma il valore è ancora molto basso. Dunque le probabilità di vincere la scommessa sono ancora molto basse.

## Regressione logistica

Nell'esempio precedente, alcune delle variabili indipendenti erano numeriche e altre erano categoriche. Ma la variabile dipendente era sempre numerica.

La regressione lineare può essere generalizzata per gestire anche altri tipi di variabili dipendenti.
Se la varibile dipendente è booleana, il modello si chiamerà regressione logistica. Se il modello è in intero averemo un regressione di Poisson.

Come esempio di regressione logistica, consideriamo una variazione della scommessa. Supponiamo di voler predire il sesso del nascituro. Possiamo provare ad usare i dati del dataset per trovare i fattori che influenzano il "rapporto tra i sessi", che convenzionalemente definisce la probabilità di avere un maschietto.

Se codifichiamo la variabile dipendente numericamente per esempio impostiamo 0 per una bambina e 1 per un bambino possiamo usare il metodo dei minimi quadrati, ma avremmo dei problemi. Il modello lineare assomiglierà ad una cosa del genere:

$$
y = \beta_0 + \beta_1 x1 + \beta_2 x2 + \varepsilon
$$


Dove $y$ rappresenta la variabile dipendente e $x1$, $x2$ sono le variabili indipendenti, troveremo poi i parametri che minimizzano i residuali.

Il problema con questo approccio è che produce una previsione difficile da interpretare, Dato i parametri stimati e i valori di $x1$ e $x2$ il modello potrebbe restiruire come risultato $y=0.5$, ma ovviamente gli unici valori significativi di y sono 0 e 1.

Potremmo essere tentati a interpretare il risultato come una probabilità, per esempio potremmo dire che il valore sopra rappresenta il 50% di probabilità di avere un ragazzo. Ma il modello potrebbe tranquillamente dare come risultato $y=1.1$ o $y=-0.1$ e queste ovviamente non sono probabilità valide.

La regressione logistica evita questo problema esprimento la previzione in termini di quote invece che di probabilità.
Se non si e familiare con le quote, "quote a favore" di un evento è il rapporto tra la probabilità che questo evento avvenga e la probabibilità che questo non avvenga.

Per fare un esempio se ho il 75% di probabilita di vittoria, Sto semplicemente dicendo che le quote in nostro favore sono tre a uno perchè le possibilità di vincere sono tre volte la possibilità di perdere.

Le quote le probabilità sono due rappresentazioni differenti della stessa informazione. Data una probabilità posso calcolare le quote semplicemente usando la formula:

$$
o = \frac{p}{(1-p)}
$$

mentre per tornare alla probabilità si usa semplicemente funzione inversa:

$$
p = \frac{o}{(o+1)}
$$

La regressione logistica è basata sul modello seguente:

$$
    log(o) = \beta_0 + \beta_1 x1 + \beta_2 x2 + \varepsilon 
$$


Dove o rappresenta la quota a nostro favore di un particolare esito, nel nostro caso è la quota a nostro favore di avere un ragazzo.
Supponiamo di avere stimato i parametri $\beta_0$, $\beta_1$, $\beta_2$ e supponiamo di avere i valori di $x1$ e $x2$.
Possiamo cosi avere i valori di log(o) e convertirli in probabilità con il seguente codice:

```python

o = np.exp(log_o)
p = o / (o+1)

```

Possiamo dunque calcolare probabilità, ma come possiamo stimare i parametri?

## Stima dei parametri
Sfortunatamente la regressione logistica non ha una soluzione a forma chiusa, il problema deve essere risolto dando una soluzione iniziale e provvedere a migliorarla iterativamente.
L'obiettivo più comunune è quello di trovare il maximum-likelihood estimate (MLE) che rappresenta l'insieme dei parametri che massimizza la probabilità dei dati, supponiamo di avere i seguenti dati:

In [62]:
y = np.array([0, 1, 0, 1])
x1 = np.array([0, 0, 0, 1])
x2 = np.array([0, 1, 1, 1])

Partiamo con i seguenti parametri iniziali $\beta_0=−1.5$, $\beta_1=2.8$ e $\beta_2=1.1$:
Per ogni riga possiamo calcolare `log_o`

In [63]:
beta = [-1.5, 2.8, 1.1]
log_o = beta[0] + beta[1] * x1 + beta[2] * x2 
log_o

array([-1.5, -0.4, -0.4,  2.4])

Convertiamo poi il rapporto in probabilità:

In [64]:
o = np.exp(log_o)
o

array([ 0.22313016,  0.67032005,  0.67032005, 11.02317638])

In [65]:
p = o / (o+1)
p

array([0.18242552, 0.40131234, 0.40131234, 0.9168273 ])

Si noti che quando `log_o` e maggiore di 0 consegue che `o` sarà maggiore di 1 e `p` sarà maggiore di 0.5.
Per calcolare il valore di mle usiamo p quando il valore di y sarà 1 altrimenti 1-p quando il valore di y sarà 0.
Facciamo un esempio se il modello restituisce una probabilità di un bambino pari a 0.8 se y rappresenta un bambino il risultato della nostra funzione sarà 0.8 altrimenti se y rappresenta una bambina il risultato sarà 0.2 
Possiamo calcolare questa probabilità in questo modo:

In [66]:
likes = y * p + (1-y) * (1-p)
likes

array([0.81757448, 0.40131234, 0.59868766, 0.9168273 ])

La probabiità complessiva è il prodotto delle singole probabilità

In [67]:
like = np.prod(likes)
like

0.1800933529673034

Per questi parametri di beta, la probabilità complessiva è pari a 0.18. L'obiettivo della regressione logistica è quello di trovare i parametri che massimizzano tale valore. Per farlo i più importanti software di statistica usano il risolutori tipo il metodo di Newton.

## Implementazione
StatsModels fornisce una implementazione della regressione logistica chiamata logit. 
Per dimostrare come usarlo, cercheremo di vedere quali variabili influicono col rapporto del sesso del nascituro.

In [68]:
live['boy'] = (live.babysex==1).astype(int)

I fattori che potrebbero influenzare il sesso del nascituro includono l'età della madre, la razza e lo status sociale.
Partiamo analizzando l'età della madre:

In [69]:
import statsmodels.formula.api as smf

model = smf.logit('boy ~ agepreg', data=live)
results = model.fit()

Optimization terminated successfully.
         Current function value: 0.693022
         Iterations 3


`logit` funziona allo stesso modo di `ols` con la stessa formula scritta in Patsy. Il risultato è un oggetto di tipo `Logit` che rappresenta il modello.
L'oggetto contiene degli attributi chiamati endog e exog che contiene le variabili endogene, un altro nome delle variabili dipendenti e le variabili esogene un altro nome per le variabili indipendenti.

Visto che il risultato in Numpy, a volte è conveniente convertirlo in un DataFrames:

In [70]:
import pandas as pd

endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

Il risultato di `model.fit` è un oggetto di tipo `BinaryResults` molto simile al tipo già visto `RegressionResults` questo è il risultato:

In [71]:
summarizeResults(results)

Intercept   0.00669   (0.946)
agepreg   0.000982   (0.798)
R^2 5.361e-06


Il parametro `agepreg` è positivo il che suggerisce che le madri più anziane hanno più probabilità ad avere dei maschietti ma il valore di p-value è pari a 0.783 che significa che l'effetto apparente è facilmente dovuto al caso.

Il coefficiente di determinazione $R^2$ non si applica alla regressione logistica, ma ci sono molte alternative che possono essere usate come uno pseudo valore $R^2$.
Questo valore può essere utile per comparare i modelli, per esempio abbiamo creato un modello che include anche altri fattori:

In [72]:
formula = 'boy ~ agepreg + hpagelb + birthord + C(race)'
model = smf.logit(formula, data=live)
results = model.fit()

Optimization terminated successfully.
         Current function value: 0.692727
         Iterations 4


Assieme all'età della madre il modello include l'età del padre `hpagelb` e l'ordine di nascita `birthord` il campo `race` è stato codificato come una variabile categorica, qui i risultati:

In [73]:
summarizeResults(results)

Intercept   -0.0259   (0.801)
C(race)[T.2]   -0.0303   (0.553)
C(race)[T.3]   -0.00638   (0.939)
agepreg   -0.00339   (0.463)
hpagelb   0.00483   (0.0435)
birthord   0.013   (0.558)
R^2 0.000431


Nessuno dei parametri è statisticamente significativo, il valore di pseudo-$R^2$ è leggermente più grande, ma questo potrebbe essere dovuto al caso.

## Accuratezza

Nello scenario della scommessa d'ufficio, siamo interessati alla accuratezza del modello: il numero di previsioni corrette, rispetto a quello che avremo per puro caso. 
Nel dataset NSFG ci sono più bambini che bambine, dunque la strategia di base potrebbe essere quella di restituire sempre come risultato il valore "bambino". L'accuratezza di questa strategia è la frazione dei ragazzi:

In [74]:
actual = endog['boy']
baseline = actual.mean()

Visto `actual` è codificato come un campo intero, la media rappresenta la frazione dei ragazzi che è pari a:

In [75]:
baseline

0.5078009338344153

Calcoliamo l'accuratezza del modello:

In [76]:
predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)

`results.predict` restituisce un array Numpy di probabilità, che arrontonderemo a valori di 0 e 1.
Moltiplicando i valori attuali con le previsioni otteniamo il numero di previsione dei ragazzi corrette, che chiamaremo veri positivi.
Similmente per i valori veri negativi `true_neg` indicheremo il numero delle previsioni delle ragazze corrette. 
L'Accuratezza rappresenta la frazione delle previsioni corrette:

In [77]:
acc = (sum(true_pos) + sum(true_neg)) / len(actual)
acc

0.516000455528983

Il risultato è leggermente migliore del valore baseline. 
Ma questo valore non deve essere preso troppo seriamente. 
Abbiamo usato gli stessi dati per costruire e testare il modello, dunque il modello potrebbe non evere molto potere previsionale sui nuovi dati.

Tuttavia, useremo il modello per eseguire la previsione della nostra scommessa dell'ufficio.
Supponiamo che la nostra amica abbia 35 anni sia bianca e il suo marito abbia 39 anni e che stiano aspettando il terzo figlio:

In [78]:
columns = ['agepreg', 'hpagelb', 'birthord', 'race']
new = pd.DataFrame([[35, 39, 3, 2]], columns=columns)
results.predict(new)

0    0.513137
dtype: float64

Il modello restituisce il valore 0.51 il che indica che con poca probabilità avremo un bambino.
Il modello aumenta la possibilità di vincita ma con poca differenza dal modello base.